# Master Thesis First Version of the STT Model 

**Author**: Karin Thommen

**Date**: March 2023 


---

**Content of the Notebook**: Train and test STT model (Spontaneous and prepared Speech) 

---

In [ ]:
# references: 
# https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLS_R_on_Common_Voice.ipynb#scrollTo=72737oog2F6U 
# 
# 

## Import and Setup

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 31 05:02:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install torch
!pip install datasets
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer
!pip install audio-metadata

In [3]:
pip install "dill<0.3.5"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 KB 10.9 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.4 which is incompatible.


In [ ]:
#!pip install git+https://github.com/huggingface/transformers

In [4]:
import pandas as pd
import os
import transformers
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

from datasets.fingerprint import Hasher
import pickle
import dill

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json

import IPython.display as ipd
import numpy as np
import random

import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import audio_metadata

from datasets import load_dataset, Audio, load_metric
from datasets import Dataset

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [5]:
# Build connection to data folder on GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# load train, test and validation files 
train=pd.read_csv("/content/drive/MyDrive/Masterarbeit/Data/SDS-200/SDS-200 Corpus/splits/train.tsv",sep='\t')
test=pd.read_csv("/content/drive/MyDrive/Masterarbeit/Data/SDS-200/SDS-200 Corpus/splits/test.tsv",sep='\t')
valid=pd.read_csv("/content/drive/MyDrive/Masterarbeit/Data/SDS-200/SDS-200 Corpus/splits/valid.tsv",sep='\t')

# Train Model on Prepared Speech

## Prepare Data

In [7]:
# show training data 
train.head()

,clip_id,clip_path,sentence,clip_created_at,clip_is_valid,sentence_id,sentence_source,client_id,zipcode,canton,user_mean_clip_quality,clip_n_votes_correct,clip_n_votes_false,clip_n_times_reported,sentence_n_times_reported,age,gender,duration,user_sentences,continuous_client_index
0,127972,8587421a-f201-4e8a-95a9-e3d3f2473f5e/0f97ad53f...,Dies wird inzwischen durch Forschungsergebniss...,2021-08-25 08:20:01,NaN,0f97ad53f8c704aa2d9b0200bcf5718bacae8b58128106...,cv_sentences,8587421a-f201-4e8a-95a9-e3d3f2473f5e,5000.0,AG,0.9011,0,0,0,0,fourties,male,4.176,1038,NaN
1,32935,b5685887-4f42-40dd-8e46-349bcb5ff4aa/034cb4228...,Des Weiteren gibt es noch eine Abteilung für E...,2021-07-11 13:51:12,True,034cb4228965e95835de3cf62c129b148e5425f704a1ba...,cv_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,2,0,0,0,thirties,male,3.960,11696,16.0
2,124203,b5685887-4f42-40dd-8e46-349bcb5ff4aa/2c5681ae9...,Und wo ich jetzt bin.,2021-08-24 14:24:04,NaN,2c5681ae96fe29fb5db2e1bfb523bb9e413a05d2dbec87...,tamedia_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,0,0,0,0,thirties,male,2.124,11696,NaN
3,85001,956d47bc-29f0-4ed7-ad31-f39df394f945/1e3d7a806...,Gegen Stuttgart 21 wird seit Monaten heftig pr...,2021-08-03 08:55:54,NaN,1e3d7a8060fc453c110d28aa0e2abe2031cf34d0ad5e3e...,tamedia_sentences,956d47bc-29f0-4ed7-ad31-f39df394f945,8810.0,ZH,0.9764,0,0,0,0,fourties,male,5.040,13365,NaN
4,113554,b5685887-4f42-40dd-8e46-349bcb5ff4aa/27f4be70b...,Zu mehr Freiheit wird die neue Religion aber n...,2021-08-20 19:50:09,NaN,27f4be70b7e7134e9a65ff9ec14b27577a9f4b340d8022...,tamedia_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,0,0,0,0,thirties,male,4.212,11696,NaN


In [8]:
# Currently, on spontaneous speech, only data on ZH dialect is available. 
# -> Make two train, test and validation sets: one with all dialects and one with only ZH for a better comparability 
train_all = train.copy()
train_all = train_all.sample(n=10000)
train_ZH = train[train.canton == "ZH"] # contains only ZH dialect 
test_ZH = test[test.canton == "ZH"]
valid_ZH = valid[valid.canton == "ZH"]
train_ZH.head()

,clip_id,clip_path,sentence,clip_created_at,clip_is_valid,sentence_id,sentence_source,client_id,zipcode,canton,user_mean_clip_quality,clip_n_votes_correct,clip_n_votes_false,clip_n_times_reported,sentence_n_times_reported,age,gender,duration,user_sentences,continuous_client_index
1,32935,b5685887-4f42-40dd-8e46-349bcb5ff4aa/034cb4228...,Des Weiteren gibt es noch eine Abteilung für E...,2021-07-11 13:51:12,True,034cb4228965e95835de3cf62c129b148e5425f704a1ba...,cv_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,2,0,0,0,thirties,male,3.960,11696,16.0
2,124203,b5685887-4f42-40dd-8e46-349bcb5ff4aa/2c5681ae9...,Und wo ich jetzt bin.,2021-08-24 14:24:04,NaN,2c5681ae96fe29fb5db2e1bfb523bb9e413a05d2dbec87...,tamedia_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,0,0,0,0,thirties,male,2.124,11696,NaN
3,85001,956d47bc-29f0-4ed7-ad31-f39df394f945/1e3d7a806...,Gegen Stuttgart 21 wird seit Monaten heftig pr...,2021-08-03 08:55:54,NaN,1e3d7a8060fc453c110d28aa0e2abe2031cf34d0ad5e3e...,tamedia_sentences,956d47bc-29f0-4ed7-ad31-f39df394f945,8810.0,ZH,0.9764,0,0,0,0,fourties,male,5.040,13365,NaN
4,113554,b5685887-4f42-40dd-8e46-349bcb5ff4aa/27f4be70b...,Zu mehr Freiheit wird die neue Religion aber n...,2021-08-20 19:50:09,NaN,27f4be70b7e7134e9a65ff9ec14b27577a9f4b340d8022...,tamedia_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,0,0,0,0,thirties,male,4.212,11696,NaN
5,124119,956d47bc-29f0-4ed7-ad31-f39df394f945/0f1e14cda...,Sie hat bisher alle Weltmeisterschaften gewonnen.,2021-08-24 14:13:08,NaN,0f1e14cdaad422d35523e043a55721764d5dcd8f3767df...,cv_sentences,956d47bc-29f0-4ed7-ad31-f39df394f945,8810.0,ZH,0.9764,0,0,0,0,fourties,male,3.384,13365,NaN


In [13]:
train = train[train['duration'] <= 5.0]
train = train.reset_index()
train = train.drop(columns=["index"])
train = train.sample(n=10000)
len(train)

10000

In [14]:
# check if sentences are clean or if there is need of some preprocessing 
# reference: https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLS_R_on_Common_Voice.ipynb#scrollTo=72737oog2F6U 
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(train["sentence"], num_examples=10)

,sentence
3199,Das ist der Hauptgrund.
8842,Auch Ricardo Rodriguez interessiert das nicht.
4336,Und auf alles andere einen Scheiss geben.
2362,Sie sollen den Abend einfach geniessen.
2678,Auch ein Vertreter von Geschädigten wehrte sich gegen eine Verzögerung.
1085,Über 20 Kinder sind ums Leben gekommen.
117,Wer sich dahinter alles versteckt?
2286,Es sollte nicht das letzte Mal gewesen sein.
1385,Einzelne Spieler haben individuelle Fortschritte gemacht.
3907,Mit dem Beitrag möchte sie Anwohner warnen.


### Remove Special Characters and clean all datasets

In [15]:
# Remove special characters 
chars = '[\'̈\’\•\‹\₂\›\–\²\½\‑\°\`\&\(\)\*\+\/\=\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_chars(data):
  data["sentence"] = data['sentence'].str.replace(chars, '', regex=True)
  return data

# do for all datasets 
train_clean = remove_special_chars(train)
valid_clean = remove_special_chars(valid)
test_clean = remove_special_chars(test)

### Get Vocabulary

In [16]:
def extract_chars(data):
  all_text = " ".join(data["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

# extract the characters from all datasets 
train_vocab = extract_chars(train_clean)
valid_vocab = extract_chars(valid_clean)
test_vocab = extract_chars(test_clean)

# make a vocabulary list out of all three data sets and enumerate all characters and save them enumerated in a dictionary 
vocab_list = list(set(train_vocab["vocab"][0]) | set(valid_vocab["vocab"][0]) | set(test_vocab["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict


{' ': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 'A': 11,
 'B': 12,
 'C': 13,
 'D': 14,
 'E': 15,
 'F': 16,
 'G': 17,
 'H': 18,
 'I': 19,
 'J': 20,
 'K': 21,
 'L': 22,
 'M': 23,
 'N': 24,
 'O': 25,
 'P': 26,
 'Q': 27,
 'R': 28,
 'S': 29,
 'T': 30,
 'U': 31,
 'V': 32,
 'W': 33,
 'X': 34,
 'Y': 35,
 'Z': 36,
 'a': 37,
 'b': 38,
 'c': 39,
 'd': 40,
 'e': 41,
 'f': 42,
 'g': 43,
 'h': 44,
 'i': 45,
 'j': 46,
 'k': 47,
 'l': 48,
 'm': 49,
 'n': 50,
 'o': 51,
 'p': 52,
 'q': 53,
 'r': 54,
 's': 55,
 't': 56,
 'u': 57,
 'v': 58,
 'w': 59,
 'x': 60,
 'y': 61,
 'z': 62,
 '\xad': 63,
 'Ä': 64,
 'Ö': 65,
 'Ü': 66,
 'à': 67,
 'ä': 68,
 'ç': 69,
 'é': 70,
 'ö': 71,
 'ü': 72}

In [17]:
# this cell only once after the above cell. Do not rerun this cell without rerunning the above cell. 

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "] # replace the empty string in the vocabulary with a character that is better visible 
vocab_dict["[UNK]"] = len(vocab_dict) # add unknown token 
vocab_dict["[PAD]"] = len(vocab_dict) # add padding token or blank token (important for CTC algorithm)
print("Length of the vocabulary:", len(vocab_dict))

Length of the vocabulary: 75


In [18]:
# save vocabulary file for later usage 
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

### Load tokenizer, feature extractor and processor

In [ ]:
# load processor and model from pretrained Wav2Vec model from huggingface
#processor = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-xls-r-300m")
#model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-xls-r-300m")

In [19]:
# load vocabulary with Wav2Vec Tokenizer 
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
# instantiate feature extractor from Wav2Vec2 Feature Extractor with sampling rate of 16kHz. 
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [21]:
# after loading the tokenizer and the feature extractor, both get wrapped in a processor 
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Prepare audio clips

In [22]:
# checking the meta data of the audio to get the sampling rate of the saved audios 
path_all = "/content/drive/MyDrive/Masterarbeit/Data/SDS-200/export_20211220_clips-001/"
metadata = audio_metadata.load(path_all + train["clip_path"][0])
print(metadata)

<MP3({
    'filepath': '/content/drive/MyDrive/Masterarbeit/Data/SDS-200/export_20211220_clips-001/88f9c51c-ed4d-4c4e-b929-f2b225dc0b96/094552180b7bf5415b7b9c2dc9224d12b0607c4952ef3d6e4f1a8c21efc5e205.mp3',
    'filesize': '21.65 KiB',
    'pictures': [],
    'streaminfo': <MP3StreamInfo({
        'bitrate': '48 Kbps',
        'bitrate_mode': <MP3BitrateMode.CBR>,
        'channel_mode': <MP3ChannelMode.MONO>,
        'channels': 1,
        'duration': '00:04',
        'layer': 3,
        'protected': False,
        'sample_rate': '32.0 KHz',
        'version': 1,
    })>,
    'tags': <ID3v2Frames({
        'encodersettings': ['Lavf58.29.100'],
        'usertext': [
            <ID3v2UserText({'description': 'major_brand', 'text': ['iso5']})>,
            <ID3v2UserText({'description': 'minor_version', 'text': ['1']})>,
            <ID3v2UserText({'description': 'compatible_brands', 'text': ['isomiso5hlsf']})>,
        ],
    })>,
})>


In [ ]:
import librosa

In [ ]:
# metadata of audio shows that the clips are saved with a sampling rate of 32kHz. Convert all to 16kHz 
# First: Load audio clips to Dataset object 

# convert clip path to list (for all three datasets)
def make_audio_dataset(data):
  audio_list = data.clip_path.values.tolist() # convert the column to a list 
  audio_list = [path_all + s for s in audio_list] # add root of path to all pathes in list 
  sentences = data.sentence.values.tolist()

  resampled_audio_list = []

  for a in audio_list: 
    resample, sr = librosa.load(a, sr=16000)
    res_array = np.asarray(resample, dtype="float32")
    resampled_audio_list.append({'array': res_array, "path": a, "sampling_rate": 16000})

  #audio_dataframe = pd.DataFrame.from_dict({"path": data.clip_path.values.tolist(), "audio": resampled_audio_list, "sentence": data.sentence.values.tolist()})

  audio_dataset = Dataset.from_dict({"path": data.clip_path.values.tolist(), "audio": audio_list, "sentence": data.sentence.values.tolist()}).cast_column("audio", Audio(sampling_rate=16_000)) # convert it to a Dataset for later usage 
  
  audio_dataframe = pd.DataFrame(audio_dataset)
  print(audio_dataframe)
  print(type(audio_dataframe))

  audio_dataframe['input_values'] = audio_dataframe['audio']
  audio_dataframe['input_length'] = 0

  for index, row in audio_dataframe.iterrows():
      audio = row['audio']
      value = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0] 
      audio_dataframe.at[index, "input_values"] = value 
      audio_dataframe.at[index, "input_length"] = value.size
      with processor.as_target_processor():
          row["labels"] = processor(row["sentence"]).input_ids


  return audio_dataframe


#train_audio = make_audio_dataset(train)
valid_audio = make_audio_dataset(valid[:10])
#test_audio = make_audio_dataset(test)

# check meta data via newly created Dataset object
#train_audio[0]
valid_audio.head()

                                                path  \
0  560e4ec5-899c-40be-a05c-4692fd4cf448/0354a9243...   
1  560e4ec5-899c-40be-a05c-4692fd4cf448/037138d74...   
2  560e4ec5-899c-40be-a05c-4692fd4cf448/037514508...   
3  560e4ec5-899c-40be-a05c-4692fd4cf448/0354b9edc...   
4  560e4ec5-899c-40be-a05c-4692fd4cf448/036d9f758...   
5  560e4ec5-899c-40be-a05c-4692fd4cf448/02ccb8ff5...   
6  560e4ec5-899c-40be-a05c-4692fd4cf448/02d261597...   
7  560e4ec5-899c-40be-a05c-4692fd4cf448/0359a82c6...   
8  560e4ec5-899c-40be-a05c-4692fd4cf448/02977a2bf...   
9  560e4ec5-899c-40be-a05c-4692fd4cf448/0367037ba...   

                                               audio  \
0  {'path': '/content/drive/MyDrive/Masterarbeit/...   
1  {'path': '/content/drive/MyDrive/Masterarbeit/...   
2  {'path': '/content/drive/MyDrive/Masterarbeit/...   
3  {'path': '/content/drive/MyDrive/Masterarbeit/...   
4  {'path': '/content/drive/MyDrive/Masterarbeit/...   
5  {'path': '/content/drive/MyDrive/Masterarbei

,path,audio,sentence,input_values,input_length
0,560e4ec5-899c-40be-a05c-4692fd4cf448/0354a9243...,{'path': '/content/drive/MyDrive/Masterarbeit/...,Hier ist ein Wechsel nach drei bis zehn Jahren...,"[4.324845e-05, 4.3248623e-05, 4.324871e-05, 4....",72000
1,560e4ec5-899c-40be-a05c-4692fd4cf448/037138d74...,{'path': '/content/drive/MyDrive/Masterarbeit/...,Die Nationalbank stützt diese Sicht,"[0.0003564448, 0.00035644879, 0.0003564477, 0....",73728
2,560e4ec5-899c-40be-a05c-4692fd4cf448/037514508...,{'path': '/content/drive/MyDrive/Masterarbeit/...,Dies entspricht einem Anteil von 39 Prozent,"[0.0001287942, 0.00012879386, 0.00012879352, 0...",88128
3,560e4ec5-899c-40be-a05c-4692fd4cf448/0354b9edc...,{'path': '/content/drive/MyDrive/Masterarbeit/...,Sie ging am langen Seil,"[0.00050435483, 0.0005043546, 0.00050435483, 0...",63360
4,560e4ec5-899c-40be-a05c-4692fd4cf448/036d9f758...,{'path': '/content/drive/MyDrive/Masterarbeit/...,Die Auszeichnung wird traditionell bisher nur ...,"[0.00037747697, 0.0003774767, 0.000377477, 0.0...",138240


In [ ]:
print((valid_audio[["audio"]]))

                                               audio
0  {'path': '/content/drive/MyDrive/Masterarbeit/...
1  {'path': '/content/drive/MyDrive/Masterarbeit/...
2  {'path': '/content/drive/MyDrive/Masterarbeit/...
3  {'path': '/content/drive/MyDrive/Masterarbeit/...
4  {'path': '/content/drive/MyDrive/Masterarbeit/...
5  {'path': '/content/drive/MyDrive/Masterarbeit/...
6  {'path': '/content/drive/MyDrive/Masterarbeit/...
7  {'path': '/content/drive/MyDrive/Masterarbeit/...
8  {'path': '/content/drive/MyDrive/Masterarbeit/...
9  {'path': '/content/drive/MyDrive/Masterarbeit/...


In [ ]:
rand_int = random.randint(0, len(valid_audio)-1)

print("Target text:", valid_audio[rand_int]["sentence"])
print("Input array shape:", valid_audio[rand_int]["audio"]["array"].shape)
print("Sampling rate:", valid_audio[rand_int]["audio"]["sampling_rate"])

In [23]:
def make_audio_dataset(data):
  audio_list = data.clip_path.values.tolist() # convert the column to a list 
  audio_list = [path_all + s for s in audio_list] # add root of path to all pathes in list 

  audio_dataset = Dataset.from_dict({"path": data.clip_path.values.tolist(), "audio": audio_list, "sentence": data.sentence.values.tolist()}).cast_column("audio", Audio(sampling_rate=16_000)) # convert it to a Dataset for later usage 
  
  
  return audio_dataset 


train_audio = make_audio_dataset(train)
valid_audio = make_audio_dataset(valid)
test_audio = make_audio_dataset(test)

# check meta data via newly created Dataset object
train_audio[0]

{'path': '34a9e9a2-fd64-4b8b-b5e3-6d5654faba73/3130818254fcc2173351b1da250183678088d148aab3b287ba4ef3e0566d7bac.mp3',
 'audio': {'path': '/content/drive/MyDrive/Masterarbeit/Data/SDS-200/export_20211220_clips-001/34a9e9a2-fd64-4b8b-b5e3-6d5654faba73/3130818254fcc2173351b1da250183678088d148aab3b287ba4ef3e0566d7bac.mp3',
  'array': array([ 4.88853402e-12, -4.77484718e-12,  7.16227078e-12, ...,
          2.34431354e-08, -1.26528903e-07, -4.72482498e-07]),
  'sampling_rate': 16000},
 'sentence': 'Nur weil jemand Fotos von ihnen geteilt hatte'}

In [ ]:
# Check if audio loading worked with a random audio and sentence 
rand_int = random.randint(0, len(train_clean)-1)
print(train_clean["sentence"][rand_int])
ipd.Audio(data=train_audio[rand_int]["audio"]["array"], autoplay=True, rate=16000)

Der Rest ist bekanntlich sekundär


In [24]:
rand_int = random.randint(0, len(train_audio)-1)

print("Target text:", train_audio[rand_int]["sentence"])
print("Input array shape:", train_audio[rand_int]["audio"]["array"].shape)
print("Sampling rate:", train_audio[rand_int]["audio"]["sampling_rate"])

Target text: Mein Mandant ist nicht unschuldig
Input array shape: (46656,)
Sampling rate: 16000


In [26]:
train_audio[39000]

IndexError: ignored

In [ ]:
'''
def make_splits(whole_data): 
  n = len(whole_data)
  size = n //5
  split_1 = train_audio.select([i for i in range(0, size)])
  split_2 = train_audio.select([i for i in range(size, 2*size)])
  split_3 = train_audio.select([i for i in range(2*size, 3*size)])
  split_4 = train_audio.select([i for i in range(3*size, 4*size)])
  split_5 = train_audio.select([i for i in range(4*size, 5*size)])

  return split_1, split_2, split_3, split_4, split_5

train_audio_1, train_audio_2, train_audio_3, train_audio_4, train_audio_5 = make_splits(train_audio)
print(len(train_audio_1))
print(len(train_audio_2))
print(len(train_audio_3))
print(len(train_audio_4))
print(len(train_audio_5))
'''

In [ ]:
'''
from pathlib import Path


# go through all dataset and check for not existing audio files to avoid a crash at a later point 

def existing_audio(audio_data):
  exclude_idx = []
  c = -1
  for i in audio_data: 
    c += 1 
    if c % 100 == 0: 
      print(c)

    if Path(i["audio"]["path"]).exists(): 
      continue

    else: 
      print(i)
      exclude_idx.append(c)
      c -= 1

    
  audio_data = audio_data.select((i for i in range(len(audio_data)) if i not in set(exclude_idx)))
  return audio_data



print("Check Training")
train_audio = existing_audio(train_audio)
train_audio_1 = existing_audio(train_audio_1)
train_audio_2 = existing_audio(train_audio_2)
train_audio_3 = existing_audio(train_audio_3)
train_audio_4 = existing_audio(train_audio_4)
train_audio_5 = existing_audio(train_audio_5)

print("Check Validation")
valid_audio = existing_audio(valid_audio)
print("Check Test")
test_audio = existing_audio(test_audio)
'''

In [ ]:
# train_audio_df_1 = train_audio[:10000]

In [ ]:
# train_audio_df_2 = train_audio[10000:20000]

In [ ]:
# train_audio_df_3 = train_audio[20000:30000]

In [ ]:
# train_audio_df_4 = train_audio[30000:40000]

In [ ]:
#train_audio_df = train_audio[:]

In [ ]:
#valid_audio_df = valid_audio[:]

In [ ]:
#test_audio_df = test_audio[:]

In [ ]:
def prepare_dataframe(dataset): 
  print("Load Dataset to Dataframe")
  df = pd.DataFrame(dataset)
  df['input_values'] = df['audio']
  df['input_length'] = 0

  print("Add input values and length to dataframe")
  for index, row in tqdm(df.iterrows(), total=df.shape[0]):
      audio = row['audio']
      value = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0] 
      df.at[index, "input_values"] = value 
      df.at[index, "input_length"] = value.size
      with processor.as_target_processor():
          row["labels"] = processor(row["sentence"]).input_ids

  return Dataset.from_pandas(df)

In [ ]:
valid_dataset = prepare_dataframe(valid_audio)

In [ ]:
test_dataset = prepare_dataframe(test_audio)

In [ ]:
train_dataset = prepare_dataframe(train_audio)

In [27]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [28]:
train_final = train_audio.map(prepare_dataset, remove_columns=train_audio.column_names)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [36]:
test_final = test_audio.map(prepare_dataset, remove_columns=test_audio.column_names)

Map:   0%|          | 0/3636 [00:00<?, ? examples/s]

In [37]:
valid_final = valid_audio.map(prepare_dataset, remove_columns=valid_audio.column_names)

Map:   0%|          | 0/3638 [00:00<?, ? examples/s]

In [ ]:
train_final.to_json()
train_final.save_to_disk()

In [ ]:
from google.colab import files
files.download('train_final.json')

In [ ]:
valid_dataset.to_json()
valid_dataset.save_to_disk()

### Training

In [29]:
 @dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2ForCTC
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [30]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [31]:
wer_metric = load_metric("wer")

    load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate


In [32]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [33]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.weight', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_q.bias', 'project_hid.weight', 'quantizer.codevectors', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [34]:
model.freeze_feature_extractor()

In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/Masterarbeit",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=False,
)

In [38]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=valid_final,
    eval_dataset=test_final,
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [39]:
train_audio[1]

{'path': 'b5685887-4f42-40dd-8e46-349bcb5ff4aa/281d89315f2b724b54b732d5f9c7bb4551c0e342ba40570ac418cb3381fa2424.mp3',
 'audio': {'path': '/content/drive/MyDrive/Masterarbeit/Data/SDS-200/export_20211220_clips-001/b5685887-4f42-40dd-8e46-349bcb5ff4aa/281d89315f2b724b54b732d5f9c7bb4551c0e342ba40570ac418cb3381fa2424.mp3',
  'array': array([ 4.36557457e-11,  6.91215973e-11,  1.09139364e-11, ...,
          1.37474920e-09,  3.33304229e-09, -9.15179044e-10]),
  'sampling_rate': 16000},
 'sentence': '100 Tage sind zu kurz für ein Urteil'}

In [40]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 3638
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 3420
    Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
    __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mo

Step,Training Loss,Validation Loss,Wer
400,5.450500,3.041289,0.992035
800,1.906000,1.377500,0.818498
1200,0.743300,1.386103,0.739658
1600,0.342700,1.606966,0.733658
2000,0.171100,1.780083,0.703449
2400,0.111200,1.846258,0.701519


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 3636
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/Masterarbeit/checkpoint-400
Configuration saved in /content/drive/MyDrive/Masterarbeit/checkpoint-400/config.json
Model weights saved in /content/drive/MyDrive/Masterarbeit/checkpoint-400/pytorch_model.bin
Configuration saved in /content/drive/MyDrive/Masterarbeit/checkpoint-400/preprocessor_config.json
    __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
The following columns in the evaluation set  don't have a corr

Step,Training Loss,Validation Loss,Wer
400,5.450500,3.041289,0.992035
800,1.906000,1.377500,0.818498
1200,0.743300,1.386103,0.739658
1600,0.342700,1.606966,0.733658
2000,0.171100,1.780083,0.703449
2400,0.111200,1.846258,0.701519
2800,0.079200,1.924929,0.692467


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 3636
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/Masterarbeit/checkpoint-2800
Configuration saved in /content/drive/MyDrive/Masterarbeit/checkpoint-2800/config.json
Model weights saved in /content/drive/MyDrive/Masterarbeit/checkpoint-2800/pytorch_model.bin
Configuration saved in /content/drive/MyDrive/Masterarbeit/checkpoint-2800/preprocessor_config.json
Deleting older checkpoint [/content/drive/MyDrive/Masterarbeit/checkpoint-2000] due to args.save_total_limit
    __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor div

FailedPreconditionError: ignored

## Test Model on Prepared Speech
